In [1]:
import datetime
import numpy as np
import jieba
import pickle
import re
import pandas as pd

In [2]:
starttime = datetime.datetime.now()
print(starttime.strftime('***********start %Y-%m-%d %H:%M:%S ***********'))

***********start 2020-09-23 14:39:00 ***********


In [9]:
with open('E:/workspace/simhash/data/test5.pickle', mode='rb') as file:
    wordfreq = pickle.load(file)

In [3]:
def split_words(df, title, content):
    wordList = []
    text = str(df[title])+','+str(df[content])
    sentence_seged = jieba.lcut(text)
    #sentence_seged = pseg.cut(text)
    rule = '[^/.0-9a-zA-Z\u4e00-\u9fa5]+'
    for x in sentence_seged:
        word = re.sub(rule, '', x)
        if word:
            wordList.append(word)
    return ' '.join(wordList[:int(len(wordList)*0.8)])

In [4]:
def vectorize_text(text_words):
    features = text_words.split(' ')
    v = np.array([0] * 64)
    i = 0
    for f in features:
        if f in wordfreq:
            v = v + wordfreq[f]
            i = i + 1
    return v/i

In [20]:
def bit_vec(row):
    vec = [ '0' if i <= 0 else '1' for i in row['vec']]
    return tuple([''.join(vec[i*16:i*16+16]) for i in range(4)])

In [7]:
handle_data = pd.read_excel('C:/Users/yangwenbin/Downloads/query-hive-48357.xlsx', usecols = 'C, H, N, P, Q')
handle_data.rename(columns={'info_browser_tmp_sea.news_id':'news_id', 'info_browser_tmp_sea.title':'title',
                            'info_browser_tmp_sea.news_tag':'news_tag', 'info_browser_tmp_sea.type':'type',
                            'info_browser_tmp_sea.content':'content'}, inplace = True)
handle_data['text_words'] = handle_data.apply(split_words, axis=1, args=('title', 'content'))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\yangwenbin\AppData\Local\Temp\jieba.cache
Loading model cost 0.694 seconds.
Prefix dict has been built succesfully.


In [10]:
title_data = handle_data[handle_data['text_words'] != '']
title_data['vec'] = title_data['text_words'].apply(vectorize_text)
title_data[['vec0','vec1','vec2','vec3']] = title_data.apply(bit_vec, axis=1, result_type="expand")

In [122]:
wordfreq0 = dict()
wordfreq1 = dict()
wordfreq2 = dict()
wordfreq3 = dict()
for tup in zip(title_data['vec0'], title_data['vec1'], title_data['vec2'], title_data['vec3'], title_data['news_id']):
    hash_str = ''.join(tup[:-1])
    if tup[0] not in wordfreq0:
        wordfreq0[tup[0]] = []
    wordfreq0[tup[0]].append((hash_str, tup[-1]))
    if tup[1] not in wordfreq1:
        wordfreq1[tup[1]] = []
    wordfreq1[tup[1]].append((hash_str, tup[-1]))
    if tup[2] not in wordfreq2:
        wordfreq2[tup[2]] = []
    wordfreq2[tup[2]].append((hash_str, tup[-1]))
    if tup[3] not in wordfreq3:
        wordfreq3[tup[3]] = []
    wordfreq3[tup[3]].append((hash_str, tup[-1]))

In [56]:
def distance(one, two):
    one = int(one, 2)
    two = int(two, 2)
    x = (one ^ two) & ((1 << 64) - 1)
    ans = 0
    while x:
        ans += 1
        x &= x - 1
    return ans

In [136]:
names = locals()
def simlar(row, i):
#     if i == 0:
#         wordfreqi = wordfreq0
#     elif i == 1:
#         wordfreqi = wordfreq1
#     elif i == 2:
#         wordfreqi = wordfreq2
#     else:
#         wordfreqi = wordfreq3
    wordfreqi = names['wordfreq'+str(i)]
#     exec('wordfreqi = wordfreq{}'.format(i))
    if len(wordfreqi[row['vec'+str(i)]]) > 1:
        hash_id_s = wordfreqi[row['vec'+str(i)]]
        hash_str = ''.join((row['vec0'], row['vec1'], row['vec2'], row['vec3']))
        for hash_id in hash_id_s:
            if row['news_id'] < hash_id[-1]:
                if distance(hash_str, hash_id[0]) <= 3:
                    return hash_id[-1]
    return False

In [144]:
title_data[title_data['iskeep'] == True].shape
# [['news_id', 'title', 'iskeep']]

(44541, 12)

In [137]:
def iskeep(row):
    simlarid = simlar(row, 0)
    if simlarid:
        return simlarid
    else:
        simlarid = simlar(row, 1)
        if simlarid:
            return simlarid
        else:
            simlarid = simlar(row, 2)
            if simlarid:
                return simlarid
            else:
                simlarid = simlar(row, 3)
                if simlarid:
                    return simlarid
                else:
                    return True

In [140]:
title_data['iskeep'] = title_data.apply(iskeep, axis=1)

In [11]:
endtime = datetime.datetime.now()
print(endtime.strftime('***********stop %Y-%m-%d %H:%M:%S ***********'))
print((endtime - starttime).seconds)

***********stop 2020-09-23 14:47:49 ***********
529


In [74]:
#     if len(wordfreq0[row['vec0']]) > 1:
#         hash_id_s = wordfreq0[row['vec0']]
#         for hash_id in hash_id_s:
#             if row['news_id'] != hash_id[-1]:
#                 hash_str = ''.join((row['vec0'], row['vec1'], row['vec2'], row['vec3'))
#                 if distance(hash_str, hash_id[0]) <= 3:

Index(['news_id', 'title', 'news_tag', 'type', 'content', 'text_words'], dtype='object')

In [30]:
wordfreq0['1111011000100101']

[('1111011000100101110110010011110001101110000101011100101010110101',
  '2020050cfbd729cdef06ba8bd1a7da7d35377a'),
 ('1111011000100101110000010011010001001100001001011100011011111101',
  '2020050dc8e717afc94f34e500f945cc0f8abb'),
 ('1111011000100101110110010010110001100110010011011100000010000101',
  '202005dc1af5aab4f9e1b503203dc5adc5dbb6'),
 ('1111011000100101110110010010110001101110100001011100111010111101',
  '20200553c99fe4fb25773b1c4312c56bb3d712'),
 ('1111011000100101110100010010110000101110100001011100101010100000',
  '20200507bd1ab59b294778ff1b8a2e079b57ac'),
 ('1111011000100101110000010011110000001010011111011100111010110101',
  '202005da6d95a44547be49a8dd0343bd661183'),
 ('1111011000100101110000010011010001001110000001011100101011110101',
  '202005204e179d09e6ba6f6d36e9f87eb153a7'),
 ('1111011000100101110110010011010001101110100101011100011010110000',
  '202005d92b44b8f6c173f7b73c860807caa359'),
 ('1111011000100101110000010011010001001110000101011100001111110101',
  '2020059